In [184]:
import pandas as pd
import numpy as np
import os

## En datos/properati deben estar todos los archivos de ventas provistos por properati

In [185]:
filesProperati = filter( lambda f: not f.startswith('.'), os.listdir("datos/properati"))

In [186]:

dataframes = []
#Este diccionario es para luego eliminar segun fechas erroneas
for file_name in filesProperati:
    
    newDataFrame = pd.read_csv("datos/properati/"+file_name)
    dataframes.append(newDataFrame)

       

## ¿Como filtramos por las zonas pedidas?

In [193]:
nulosStateName = 0
nulosPlaceWParentNames = 0
noCuentaConStateName = 0
noCuentaConPlaceWParentNames = 0
for df in dataFrames:
    try:
        nulosStateName += df.state_name.isnull().sum()
    except:
        noCuentaConStateName += 1
    try:
        nulosPlaceWParentNames += df.place_with_parent_names.isnull().sum()
    except:
        noCuentaConPlaceWParentNames += 1
print "Cantidad de registros con valor nulo en state_name: "+str(nulosStateName)
print "Cantidad de registros con valor nulo en place_w_parent_names: "+str(nulosPlaceWParentNames)
print "Cantidad de archivos que no cuentan con la columna state_name: "+str(noCuentaConStateName)
print "Cantidad de archivos que no cuentan con la columna place_w_parent_names: "+str(noCuentaConPlaceWParentNames)

Cantidad de registros con valor nulo en state_name: 0
Cantidad de registros con valor nulo en place_w_parent_names: 0
Cantidad de archivos que no cuentan con la columna state_name: 26
Cantidad de archivos que no cuentan con la columna place_w_parent_names: 0


## Dado los resultados, para quedarnos solo con las propiedades de Capital Federal y GBA tendremos que utilizar la columna 'place_with_parent_names'

In [194]:
frecuencias = {}
for df in dataFrames:
    parentNames = df['place_with_parent_names']
    for padres in parentNames:
        zona = padres.split("|")[2]
        if zona not in frecuencias:
            frecuencias[zona] = 1
        else:
            frecuencias[zona]+=1

In [195]:
for zona in frecuencias: 
    print zona,frecuencias[zona]

Neuquén 3073
Córdoba 28847
Chaco 303
Mendoza 2163
Chubut 612
Capital Federal 172817
San Luis 2424
Tucumán 1220
Misiones 2274
Tierra Del Fuego 484
Bs.As. G.B.A. Zona Oeste 92161
Corrientes 826
Formosa 254
Río Negro 6387
Santa Cruz 84
Montevideo 3
La Pampa 1002
Buenos Aires Costa Atlántica 47635
Buenos Aires Interior 15792
Bs.As. G.B.A. Zona Sur 50812
Santiago Del Estero 25
Jujuy 100
Santa Fe 32650
Salta 2286
La Rioja 52
Entre Ríos 5574
Bs.As. G.B.A. Zona Norte 142283
Catamarca 114
San Juan 116


## Interesan: Bs.As. G.B.A. Zona Norte, Bs.As. G.B.A. Zona Sur, Bs.As. G.B.A. Zona Oeste, Capital Federal


In [196]:
def filtrarCapitalYGBA(serie):
    deseados = ["Bs.As. G.B.A. Zona Norte", "Bs.As. G.B.A. Zona Sur", "Bs.As. G.B.A. Zona Oeste", "Capital Federal"]
    booleans = []
    for item in serie:
        if item.split("|")[2] in deseados:
            booleans.append(True)
        else:
            booleans.append(False)
    return booleans

In [197]:
filtrados =[]
for df in dataFrames:
    
    df = df[filtrarCapitalYGBA(df['place_with_parent_names'])]
    filtrados.append(df)

### Por las dudas chequeo que ahora las frecuencias sean las mismas y solo queden las zonas que interesan

In [198]:
frecuencias = {}
for df in filtrados:
    parentNames = df['place_with_parent_names']
    for padres in parentNames:
        zona = padres.split("|")[2]
        if zona not in frecuencias:
            frecuencias[zona] = 1
        else:
            frecuencias[zona]+=1

In [199]:
for key in frecuencias:
    print key, frecuencias[key]

Capital Federal 172817
Bs.As. G.B.A. Zona Norte 142283
Bs.As. G.B.A. Zona Oeste 92161
Bs.As. G.B.A. Zona Sur 50812


# Ahora recupero datos, elimino algunas columnas, y demas.
# Primero miro como estan los datos, para ver si conviene trabajar con los precios en pesos argentinos o dolares. 

In [200]:
def a(df, column_name):
    try:
        return(df[column_name].isnull().sum(),0)
    except:
        return (0,1)

In [201]:
nulsPriceAproxLocalCurr = 0
nulsPriceAproxUsd = 0
nulsPricePerM2 = 0
nulsPricePerM2usd = 0

priceAproxLocalCurrInex = 0
priceAproxUsdInex = 0
pricePerM2Inex = 0
pricePerM2usdInex = 0

for df in filtrados:
    nuls, inex = a(df, "price_aprox_local_currency")
    nulsPriceAproxLocalCurr += nuls
    priceAproxLocalCurrInex += inex
    nuls,inex = a(df, "price_aprox_usd")
    nulsPriceAproxUsd += nuls
    priceAproxUsdInex += inex
    nuls, inex = a(df, "price_per_m2")
    nulsPricePerM2 += nuls
    pricePerM2Inex += inex
    nuls,inex = a(df, "price_usd_per_m2")
    nulsPricePerM2usd += nuls
    pricePerM2usdInex += inex

In [202]:
print str(nulsPriceAproxLocalCurr)+" Nulls priceAproxLocalCurr"
print str(nulsPriceAproxUsd)+" Nulls priceAproxUsd"
print str(nulsPricePerM2)+" Nulls Price Per M2"
print str(nulsPricePerM2usd)+" Nulls pricePerM2USD"
print str(priceAproxLocalCurrInex)+" Inexistentes priceAproxLocalCurr"
print str(priceAproxUsdInex)+" Inexistentes priceAproxUsd"
print str(pricePerM2Inex)+" Inexistentes pricePerM2"
print str(pricePerM2usdInex)+" Inexistentes pricePerM2usd"

58927 Nulls priceAproxLocalCurr
58927 Nulls priceAproxUsd
32915 Nulls Price Per M2
150676 Nulls pricePerM2USD
0 Inexistentes priceAproxLocalCurr
0 Inexistentes priceAproxUsd
26 Inexistentes pricePerM2
0 Inexistentes pricePerM2usd


# Tambien miro los datos acerca de las superficies

## Observo que algunos archivos solo tienen una columna 'surface_in_m2', no distingue entre si es cubierta o si es total, suponemos que es la superficie total. 

In [204]:
nulsSuperficieTotal = 0
for df in filtrados:
    try:
        nulsSuperficieTotal += df.surface_in_m2.isnull().sum()
    except:
        nulsSuperficieTotal += df.surface_total_in_m2.isnull().sum()
nulsSuperficieTotal

101233

# Recuperamos lo que podemos de cada registro

In [205]:
#Primero le pongo el mismo nombre a la superficie total
for df in filtrados:
    try:
        df['surface_total_in_m2']
    except:
        df.rename(columns={'surface_in_m2':'surface_total_in_m2'},inplace=True)
    

# Recupero la informacion que pueda, precios en dolares y superficie

In [227]:
from utils import recuperar_superficie,recuperar_precio_usd,recuperar_ppm2

In [229]:
for df in filtrados:
     df['surface_total_in_m2'] = df.apply(lambda row: recuperar_superficie(row),axis=1)
     df['price_aprox_usd'] = df.apply(lambda row: recuperar_precio_usd(row),axis=1)
     df['price_usd_per_m2'] = df.apply(lambda row: recuperar_ppm2(row),axis=1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


## Chequeo que se hayan recuperado datos

In [208]:
a = 0
b = 0
c = 0
for df in filtrados:
    a += df.price_aprox_usd.isnull().sum()
    b += df.price_usd_per_m2.isnull().sum()
    c += df.surface_total_in_m2.isnull().sum()

print str(a)+" nuls priceAproxUsd"
print str(c)+" nuls surface_total_in_m2"
print str(b)+" nuls pricePerM2usd"

58664 nuls priceAproxUsd
101233 nuls surface_total_in_m2
143935 nuls pricePerM2usd


## Se recuperaron datos acerca del precio aproximado en usd y el precio por m2 en usd. No se pudieron recuperar nuevos datos acerca de la superficie total

## A continuacion unifico y voy eliminando duplicados, teniendo en cuenta distintos subgrupos de columnas

In [209]:
unificacion = pd.concat(filtrados, axis=0, ignore_index=True)

In [210]:
len(unificacion)

458073

In [211]:
#Elimino los registros que tengan todas las columnas identicas
a = unificacion.drop_duplicates()

In [212]:
len(a)

409127

In [214]:
#Elimino registros con misma url de properati (exceptuando a los nulos)
b = a[(~a.duplicated(subset=['properati_url'])) | (a['properati_url'].isnull())]

In [215]:
#elimino duplicados 
len(b)

213869

In [216]:
#Si estan en el mismo lugar, tienen mismo precio y descripcion, son los mismos? 
c = b.drop_duplicates(subset=["place_with_parent_names","price_aprox_usd","property_type","surface_total_in_m2","description","created_on"])

In [217]:
d = c.drop_duplicates(subset = ["created_on","title","description","lat","lon","place_name"])

In [218]:
len(d)

186173

In [221]:
d.price.isnull().sum()

13179

In [222]:
d.price_aprox_usd.isnull().sum()

13161

In [223]:
d.price_aprox_local_currency.isnull().sum()

13179

## Elimino columnas con informacion redundantes o que no interesan

In [224]:
d.columns

Index([u'country_name', u'created_on', u'currency', u'description',
       u'expenses', u'extra', u'floor', u'geonames_id', u'id',
       u'image_thumbnail', u'lat', u'lat-lon', u'lon', u'operation',
       u'place_name', u'place_with_parent_names', u'price',
       u'price_aprox_local_currency', u'price_aprox_usd', u'price_per_m2',
       u'price_usd_per_m2', u'properati_url', u'property_type', u'rooms',
       u'state_name', u'surface_covered_in_m2', u'surface_total_in_m2',
       u'title'],
      dtype='object')

In [225]:
d.drop(["properati_url", "image_thumbnail","country_name", "lat","lon","operation","price_aprox_local_currency","price_per_m2","price","currency"], axis = 1, inplace= True)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [226]:
d.to_csv("datosCabaYGBA")